In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("C:\\Users\\SisLab\\!SOYEON\\train_df.csv")
df.head()

,id,week,center_id,city_code,region_code,center_type,op_area,meal_id,category,cuisine,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1379560,1,55,647,56,TYPE_C,2.0,1885,Beverages,Thai,136.83,152.29,0,0,177
1,1466964,1,55,647,56,TYPE_C,2.0,1993,Beverages,Thai,136.83,135.83,0,0,270
2,1346989,1,55,647,56,TYPE_C,2.0,2539,Beverages,Thai,134.86,135.86,0,0,189
3,1338232,1,55,647,56,TYPE_C,2.0,2139,Beverages,Indian,339.50,437.53,0,0,54
4,1448490,1,55,647,56,TYPE_C,2.0,2631,Beverages,Indian,243.50,242.50,0,0,40


In [3]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)
# center_type
data_cat = ohe.fit_transform(df[['center_type']])
df_center = pd.concat([df.drop(columns=['center_type']),pd.DataFrame(data_cat, columns=['center_type_' + str(col) for col in ohe.categories_[0]])], axis=1)

# category
data_cat = ohe.fit_transform(df[['category']])
df_category = pd.concat([df_center.drop(columns=['category']),pd.DataFrame(data_cat, columns=['category_' + str(col) for col in ohe.categories_[0]])], axis=1)

# cuisine
data_cat = ohe.fit_transform(df[['cuisine']])
train_df = pd.concat([df_category.drop(columns=['cuisine']),pd.DataFrame(data_cat, columns=['cuisine_' + str(col) for col in ohe.categories_[0]])], axis=1)

train_df.head()

,id,week,center_id,city_code,region_code,op_area,meal_id,checkout_price,base_price,emailer_for_promotion,...,category_Rice Bowl,category_Salad,category_Sandwich,category_Seafood,category_Soup,category_Starters,cuisine_Continental,cuisine_Indian,cuisine_Italian,cuisine_Thai
0,1379560,1,55,647,56,2.0,1885,136.83,152.29,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1466964,1,55,647,56,2.0,1993,136.83,135.83,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1346989,1,55,647,56,2.0,2539,134.86,135.86,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1338232,1,55,647,56,2.0,2139,339.50,437.53,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1448490,1,55,647,56,2.0,2631,243.50,242.50,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [4]:
from sklearn.model_selection import train_test_split

X = train_df.drop('num_orders', axis=1)
y = train_df['num_orders']

In [5]:
# train, test를 8:2로 나누기
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=1)

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_x)
X_train_transformed = pd.DataFrame(scaler.transform(train_x))
print("\nMinMaxScaler: \n", X_train_transformed.describe())


MinMaxScaler: 
                   0              1              2              3   \
count  365238.000000  365238.000000  365238.000000  365238.000000   
mean        0.500463       0.512199       0.409727       0.566405   
std         0.288605       0.288422       0.261217       0.257554   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.250648       0.263889       0.187500       0.377432   
50%         0.500666       0.520833       0.375000       0.544747   
75%         0.750459       0.763889       0.568182       0.758755   
max         1.000000       1.000000       1.000000       1.000000   

                  4              5              6              7   \
count  365238.000000  365238.000000  365238.000000  365238.000000   
mean        0.480382       0.521916       0.508245       0.381322   
std         0.251985       0.179139       0.289063       0.177079   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.15

# RandomForestRegressor 

In [8]:
# 모델 선언
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

rfr = RandomForestRegressor()

In [9]:
rfr.fit(train_x,train_y)

RandomForestRegressor()

In [10]:
test_pred = rfr.predict(test_x)
test_pred

array([ 29.44, 265.86,  69.25, ..., 253.09, 573.93,  41.1 ])

In [11]:
from sklearn.metrics import mean_squared_error
mean_squared_error(test_y, test_pred, squared = True)**0.5

152.39471572001608

In [12]:
from sklearn.metrics import r2_score
print(r2_score(test_y, test_pred))

0.8514300365413455


# Grid Search

In [13]:
from sklearn.model_selection import GridSearchCV

params = { 'n_estimators' : [10, 100],
           'min_samples_leaf' : [1, 2, 3],
           'min_samples_split' : [1, 2, 3],
          'n_jobs' : [-1, 0, 1],
          'random_state' : [180, 190, 200]
            }

In [14]:
gs_s = GridSearchCV(rfr, param_grid = params,  scoring = 'neg_mean_absolute_error')
gs_s.fit(train_x, train_y)
print("Best Parameters : ", gs_s.best_params_)
print("Best Score : ", gs_s.best_score_)

C:\Users\SisLab\anaconda3\envs\soyeon\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
450 fits failed out of a total of 810.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "C:\Users\SisLab\anaconda3\envs\soyeon\lib\site-packages\joblib\externals\loky\process_executor.py", line 436, in _process_worker
    r = call_item()
  File "C:\Users\SisLab\anaconda3\envs\soyeon\lib\site-packages\joblib\externals\loky\process_executor.py", line 288, in __call__
    return self.fn(*self.args, **self.kwargs)
  File "C:\Users\SisLab\anaconda3\envs\soyeon\lib\site-packages\joblib

Best Parameters :  {'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100, 'n_jobs': -1, 'random_state': 200}
Best Score :  -70.316605712791


In [15]:
test_gfs = gs_s.predict(test_x)
test_gfs

array([ 29.72913095, 240.76947619,  78.27610714, ..., 270.43159524,
       558.57579762,  40.79957143])

In [16]:
mean_squared_error(test_y, test_gfs, squared = True)**0.5

152.23660321980375

In [17]:
print(r2_score(test_y, test_gfs))

0.8517381651034416


# Random Search

In [18]:
from sklearn.model_selection import RandomizedSearchCV
rfg = RandomForestRegressor()

# Random Search 설정.
rs_model = RandomizedSearchCV(rfg, params)

In [19]:
rs_model.fit(train_x, train_y)

C:\Users\SisLab\anaconda3\envs\soyeon\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\SisLab\anaconda3\envs\soyeon\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\SisLab\anaconda3\envs\soyeon\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "C:\Users\SisLab\anaconda3\envs\soyeon\lib\site-packages\joblib\parallel.py", line 968, in __call__
    n_jobs = self._initialize_backend()
  Fi

RandomizedSearchCV(estimator=RandomForestRegressor(),
                   param_distributions={'min_samples_leaf': [1, 2, 3],
                                        'min_samples_split': [1, 2, 3],
                                        'n_estimators': [10, 100],
                                        'n_jobs': [-1, 0, 1],
                                        'random_state': [180, 190, 200]})

In [20]:
test_rs = rs_model.predict(test_x)
test_rs

array([ 29.44, 258.29,  75.83, ..., 259.5 , 523.32,  38.62])

In [21]:
mean_squared_error(test_y, test_rs, squared = True)**0.5

152.76754710754273

In [22]:
print(r2_score(test_y, test_rs))

0.8507021989302833
